In [1]:
from urllib.request import urlopen
from IPython.display import SVG
import matplotlib.pyplot as plt
from rdkit import Chem
from tqdm import tqdm
import pandas as pd
import xlsxwriter
import argparse
import pickle
import numpy as np
import json

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import visualizer as visualizer
import utils as utils
import handle_network as hn
import fragmentation_py as fragmentation_py
import library_downloader as library_downloader
import SiteLocator as modSite

In [2]:
libraries = {
    "GNPS-MSMLS": "https://external.gnps2.org/gnpslibrary/GNPS-MSMLS.json",
    "GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE": "https://external.gnps2.org/gnpslibrary/GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.json",
    "GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE": "https://external.gnps2.org/gnpslibrary/GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.json",
    "MIADB": "https://external.gnps2.org/gnpslibrary/MIADB.json",
    "BERKELEY-LAB": "https://external.gnps2.org/gnpslibrary/BERKELEY-LAB.json"
    # "GNPS-LIBRARY": "https://gnps-external.ucsd.edu/gnpslibrary/GNPS-LIBRARY.json"
}

In [3]:
library ="BERKELEY-LAB"
if not os.path.exists( os.path.join("../data/libraries", library)):
    url = "https://gnps-external.ucsd.edu/gnpslibrary/" + library + ".json"
    location = "../data/libraries/" + library + "/"
    library_downloader.download(url, location, 0.5, 0.1)

with open(os.path.join("../data/libraries", library, "data_dict_filtered.pkl"), "rb") as f:
    data_dict_filtered = pickle.load(f)

# load matches
with open(os.path.join("../data/libraries", library, "matches.pkl"), "rb") as f:
    matches = pickle.load(f)

# load cachedStructures_filtered
with open(os.path.join("../data/libraries", library, "cachedStructures.pkl"), "rb") as f:
    cachedStructures_filtered = pickle.load(f)

In [4]:
print (matches[1].pop())

('CCMSLIB00010108996', 'CCMSLIB00010107281')


In [5]:
print (data_dict_filtered['CCMSLIB00010105110']['Precursor_MZ'], data_dict_filtered['CCMSLIB00010114866']['Precursor_MZ'])

253.051 237.056


In [6]:
## create cashe helpers
helpers = dict()
for match in matches[1]:
    if match[0] not in helpers:
        helpers[match[0]] = []
    helpers[match[0]].append(match[1])

print(len(helpers))

6974


In [7]:
colums = ["mol1ID", "mol2ID", "mol1smile", "mol2smile", "delta_mass",
        "#_matched_peaks", "#_shifted_peaks", "#_unshifted_peaks", 
        "Closest_Max_Atom_Distance", "Count_Max", "Is_Max", "cosine", 
        "score", "helper_score", "best_score", "random_guess", "random_prob", "url"]




In [8]:
print (data_dict_filtered['CCMSLIB00010105110'].keys())
print(type(json.loads(data_dict_filtered['CCMSLIB00010105110']['peaks_json'])[0][1]))

dict_keys(['spectrum_id', 'source_file', 'task', 'scan', 'ms_level', 'library_membership', 'spectrum_status', 'peaks_json', 'splash', 'submit_user', 'Compound_Name', 'Ion_Source', 'Compound_Source', 'Instrument', 'PI', 'Data_Collector', 'Adduct', 'Scan', 'Precursor_MZ', 'ExactMass', 'Charge', 'CAS_Number', 'Pubmed_ID', 'Smiles', 'INCHI', 'INCHI_AUX', 'Library_Class', 'SpectrumID', 'Ion_Mode', 'create_time', 'task_id', 'user_id', 'InChIKey_smiles', 'InChIKey_inchi', 'Formula_smiles', 'Formula_inchi', 'url', 'annotation_history'])
<class 'float'>


In [9]:
m0, m1 = matches[1].pop()
molMol = cachedStructures_filtered[m1]
modifMol = cachedStructures_filtered[m0]
molUsi = hn.generate_usi(m1, library)
modifUsi = hn.generate_usi(m0, library)
molSmiles = data_dict_filtered[m1]['Smiles']
modifSmiles = data_dict_filtered[m0]['Smiles']
site = modSite.SiteLocator(data_dict_filtered[m1], data_dict_filtered[m0], molMol)

In [9]:
cols = ["score", "best_score", "random_guess", "random_prob", "delta_mass"]
selected = df[cols]
selected = selected.apply(pd.to_numeric)

# Bin the data by delta_mass into discrete bins
selected['delta_mass_bin'] = pd.cut(selected['delta_mass'], bins=10)

selected.groupby(selected['delta_mass_bin']).mean()

,score,best_score,random_guess,random_prob,delta_mass
delta_mass_bin,,,,,
"(11.844, 23.973]",0.624236,0.849112,0.429793,0.447761,15.002978
"(23.973, 35.983]",0.689106,0.909722,0.399434,0.403306,30.190569
"(35.983, 47.992]",0.610385,0.744186,0.613554,0.638066,43.058070
"(47.992, 60.001]",0.709444,0.750000,0.525307,0.504758,56.722600
"(60.001, 72.01]",0.745219,0.857143,0.452022,0.554862,64.673214
"(72.01, 84.02]",0.707714,0.866667,0.498320,0.467545,76.672133
"(84.02, 96.029]",0.627099,1.000000,0.140394,0.255604,85.049000
"(96.029, 108.038]",0.750000,0.750000,0.523571,0.275306,102.048250
"(108.038, 120.048]",1.000000,1.000000,0.096972,1.000000,109.993000


In [10]:
molMol = cachedStructures_filtered[m1]
modifMol = cachedStructures_filtered[m0]
molUsi = hn.generate_usi(m1, library)
modifUsi = hn.generate_usi(m0, library)
molSmiles = data_dict_filtered[m1]['Smiles']
modifSmiles = data_dict_filtered[m0]['Smiles']
site = modSite.SiteLocator(data_dict_filtered[m1], data_dict_filtered[m0], molSmiles)

In [11]:
df2 = pd.DataFrame(columns=colums)
count = 0
helperDirectory = os.path.join("../data/libraries",library,"nf_output/fragmentationtrees/")
matches_array = list(matches[1])
for match in tqdm(matches_array[0:10000]):
    try:
        m0, m1 = match
        if data_dict_filtered[m0]['Adduct'] != data_dict_filtered[m1]['Adduct'] or data_dict_filtered[m0]['Adduct'] != "M+H":
            continue
        molMol = cachedStructures_filtered[m1]
        modifMol = cachedStructures_filtered[m0]
        molUsi = hn.generate_usi(m1, library)
        modifUsi = hn.generate_usi(m0, library)
        molSmiles = data_dict_filtered[m1]['Smiles']
        modifSmiles = data_dict_filtered[m0]['Smiles']
        site = modSite.SiteLocator(data_dict_filtered[m1], data_dict_filtered[m0], molSmiles)
        modifLoc = utils.calculateModificationSites(modifMol, molMol, False)
        peak_presence_only = False
        combine = True
        # calculate score
        pre_helper = site.accuracy_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine, return_all=True)
        # print ("score is:", pre_helper['score'])
        for helper in helpers.get(m1, []):
            if helper != m0:
                helperFile = json.load(open(os.path.join(helperDirectory, helper + "_fragmentationtree.json")))
                try:
                    countUpdated = site.helper_molecule(data_dict_filtered[helper], data_dict_filtered[helper]['Smiles'], helperFile)
                    if countUpdated < 0:
                        raise Exception("helper error")
                    post_helper = site.accuracy_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine, return_all=True)
                    # if pre_helper['score'] != post_helper['score']:
                    #     print(pre_helper['score'], post_helper['score'])
                except:
                    import traceback
                    traceback.print_exc()
                    raise Exception("helper error2")
        post_helper = site.accuracy_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine, return_all=True)

        # generate random probability array 1-hot
        prob = np.zeros(site.molMol.GetNumAtoms())
        randInt = np.random.randint(0, site.molMol.GetNumAtoms())
        prob[randInt] = 1
        res2 = site.tempScore(modifLoc[0], prob, True)

        # generate random probability array distribution
        prb = np.random.rand(site.molMol.GetNumAtoms())
        prb = prb / prb.sum()
        res3 = site.tempScore(modifLoc[0], prb, True)

        # get max score
        maxScore = site.get_max_possible_score(modifLoc[0], peak_presence_only=peak_presence_only, combine=combine)
        
    # "mol1ID", "mol2ID", "mol1smile", "mol2smile", "delta_mass",
    #         "#_matched_peaks", "#_shifted_peaks", "#_unshifted_peaks", 
    #         "Closest_Max_Atom_Distance", "Count_Max", "Is_Max", "cosine", 
    #         "score", "best_score", "random_guess", "random_prob", "url"

        df2 = pd.concat([df2, pd.DataFrame([[molUsi, modifUsi,  molSmiles, data_dict_filtered[m0]['Smiles'], 
                                                         abs(float(data_dict_filtered[m0]['Precursor_MZ']) - float(data_dict_filtered[m1]['Precursor_MZ'])),
                                                         len(site.matchedPeaks), len(site.shifted),  len(site.unshifted),
                                                         pre_helper['closestMaxAtomDistance'],  pre_helper['count'],  pre_helper['isMax'], site.cosine, 
                                                        pre_helper['score'],  post_helper['score'], maxScore, res2['score'], res3['score'], 
                                                        visualizer.make_url("http://reza.cs.ucr.edu/", molUsi, modifUsi, molSmiles, modifSmiles, args=None)]], columns = colums)], ignore_index=True)
        count += 1
    except:
        # print stack trace
        import traceback
        traceback.print_exc()
        raise Exception("error 3")


  2%|▏         | 212/10000 [11:12<2:46:24,  1.02s/it] Traceback (most recent call last):
  File "/home/user/Substructure_Assignment/sharable/SiteLocator.py", line 228, in helper_molecule
    for bond in molSubStruct.GetBonds():
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_2710542/2383221216.py", line 27, in <module>
    countUpdated = site.helper_molecule(data_dict_filtered[helper], data_dict_filtered[helper]['Smiles'], helperFile)
  File "/home/user/Substructure_Assignment/sharable/SiteLocator.py", line 242, in helper_molecule
    raise Exception("error aromaticity")
Exception: error aromaticity
Traceback (most recent call last):
  File "/home/user/Substructure_Assignment/sharable/SiteLocator.py", line 228, in helper_molecule
    for bond in molSubStruct.GetBonds():
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last)

error cccc(c)-c1cc(=O)c2c(O)cc(O)cc2o1 ccc1c(=O)cc(-c2ccc(O)cc2)oc1c {'spectrum_id': 'CCMSLIB00010106136', 'source_file': 'tharwood/spectral_libraries/BERKELEY-LAB.mgf', 'task': '3dacd5cf9c8d4c3d8596c71bf3234fbd', 'scan': '3702', 'ms_level': '2', 'library_membership': 'BERKELEY-LAB', 'spectrum_status': '1', 'peaks_json': '[[57.735859,7591.000000],[60.362820,7733.000000],[61.686390,7666.000000],[67.018562,9854.000000],[74.295883,8630.000000],[92.370796,8995.000000],[94.381409,7688.000000],[119.049179,72440.000000],[121.028793,32521.000000],[129.615707,6840.000000],[145.028488,34099.000000],[145.868332,9692.000000],[153.017929,261226.000000],[153.031601,16681.000000],[163.038544,15625.000000],[171.029327,14274.000000],[180.455566,9134.000000],[185.384735,10830.000000],[229.049652,11517.000000],[243.063828,13284.000000],[247.057205,13324.000000],[254.057114,32911.000000],[269.911987,34036.000000],[270.049805,19665.000000],[271.059753,43145464.000000],[272.214905,37273.000000]]', 'splash':

Exception: error 3

In [ ]:
# Bin the data by delta_mass into discrete bins
df2['delta_mass_bin'] = pd.cut(df2['delta_mass'], bins=10)

# Calculate the average for each bin
df_grouped2 = df2.groupby('delta_mass_bin').mean().reset_index()


# Create the plot
fig = go.Figure()

# Add scatter plot for each column
columns = ["score", "best_score", "random_guess", "random_prob", "helper_score"]
for col in columns:
    fig.add_trace(go.Scatter(x=df_grouped2["delta_mass"], y=df_grouped2[col], mode='lines+markers', name=col))

# Customize the plot
fig.update_layout(
    title="Comparison of Scores",
    xaxis_title="delta_mass",
    yaxis_title="Score",
    showlegend=True,
)

# Show the plot
fig.show()


/tmp/ipykernel_2598396/1275229808.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [13]:
print(df['score'].describe())
print(df2['score'].describe())

count    601.000000
mean       0.652115
std        0.426481
min        0.000000
25%        0.188876
50%        1.000000
75%        1.000000
max        1.000000
Name: score, dtype: float64
count    601.000000
mean       0.551007
std        0.311003
min        0.000000
25%        0.263597
50%        0.513417
75%        0.716531
max        1.000000
Name: score, dtype: float64


In [ ]:
m1 = Chem.MolFromSmiles("N=C(O)CSc1nc2ncnc(O)c2n1", sanitize=False)
# make all bonds single
# for bond in m1.GetBonds():
    # bond.SetBondType(Chem.rdchem.BondType.SINGLE)
m1

In [ ]:
x1 = Chem.MolFromSmiles("COc1ccc(Cl)cc1N=C(O)CSc1nc2c(O)ncnc2n1C")
x1

In [ ]:
x1.HasSubstructMatch(m1)

In [ ]:
m2 = Chem.MolFromSmiles("Cn1c(SCCO)nc2c(O)ncnc21", sanitize=False)
m2

In [ ]:
x2 = Chem.MolFromSmiles("COc1ccccc1N=C(O)CSc1nc2c(O)ncnc2n1C")